# Day 3: Using Word Vectors for Fake News Classification

Run the below cells to get started.

In [ ]:
#@title Import Data { display-mode: "form" }
import math
import os
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from torchtext.vocab import GloVe

import pickle

import requests, io, zipfile
# Download class resources...
r = requests.get("https://www.dropbox.com/s/2pj07qip0ei09xt/inspirit_fake_news_resources.zip?dl=1")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

basepath = '.'

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

with open(os.path.join(basepath, 'train_val_data.pkl'), 'rb') as f:
  train_data, val_data = pickle.load(f)
  
print('Number of train examples:', len(train_data))
print('Number of val examples:', len(val_data))

Number of train examples: 2002
Number of val examples: 309


One potential source of information for websites is their descriptions (often called meta descriptions). These are descriptions embedded into the HTML of a webpage that describe what the website is about, so that search engines and other crawlers can use it to determine the content of a website. For example, here is the description for google.com, retrieved using the BeautifulSoup Python library for parsing HTML:

In [ ]:
def get_description_from_html(html):
  soup = bs(html)
  description_tag = soup.find('meta', attrs={'name':'og:description'}) or soup.find('meta', attrs={'property':'description'}) or soup.find('meta', attrs={'name':'description'})
  if description_tag:
    description = description_tag.get('content') or ''
  else: # If there is no description, return empty string.
    description = ''
  return description

def scrape_description(url):
  if not url.startswith('http'):
    url = 'http://' + url
  response = requests.get(url, timeout=10)
  html = response.text
  description = get_description_from_html(html)
  return description

print('Description of Google.com:')
print(scrape_description('google.com'))

Description of Google.com:
Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for.


## Instructor-Led Discussion: Scraping Website Descriptions

Play around with the below demo (running the cell normally) to live scrape the descriptions of different websites using the code above. Do you notice anything interesting or unexpected? If so, share with the class. (~3 minutes)

In [ ]:
#@title Live Website Description Scraper { display-mode: "both" }

url = "youtube.com" #@param {type:"string"}

print('Description of %s:' % url)
print(scrape_description(url))

Description of youtube.com:
Enjoy the videos and music you love, upload original content, and share it all with friends, family, and the world on YouTube.


### Bag-of-Words Model




It is easy to retrieve the descriptions for the fake and real news websites in our dataset as well. What can we do with these? We can use the approach from yesterday where we extract counts from the description for particular keywords and use these as features, but this would require us to manually select features that we think are important. What if our model automatically collected all of the most important keywords and added their counts for each website description to our feature vector? Our model could then learn feature weights for these words to help us correctly classify news websites.

This approach of automatically featurizing the counts of words in text is called the bag-of-words model. This name comes from the fact that the features do not store the order of the words, rather just their counts.

## Exercise 

Let's start by extracting the descriptions for the websites in our dataset. Use the helper function *get_description_from_html* defined above to extract all of the descriptions for the websites in the training data. The return value of the function should be a list of descriptions, in the same order as the sites in *train_data* (~10 minutes). Note that running the function will take a few minutes.

In [ ]:
def get_descriptions_from_data(data):
  # A dictionary mapping from url to description for the websites in 
  # train_data.
  descriptions = []
  for site in tqdm(data):
    ### YOUR CODE HERE ###
    descriptions.append(get_description_from_html(site[1]))
    
    ### END CODE ###
  return descriptions
  

train_descriptions = get_descriptions_from_data(train_data)
train_urls = [url for (url, html, label) in train_data]

print('\nNYTimes Description:')
print(train_descriptions[train_urls.index('nytimes.com')])


100%|██████████| 2002/2002 [02:29<00:00, 13.36it/s]


NYTimes Description:
The New York Times: Find breaking news, multimedia, reviews & opinion on Washington, business, sports, movies, travel, books, jobs, education, real estate, cars & more at nytimes.com.


## Let's get our value desciptions!

Once you have this working, call the *get_descriptions_from_data* for val to get *val_descriptions*. 

In [ ]:
val_descriptions = get_descriptions_from_data(val_data)


100%|██████████| 309/309 [00:22<00:00, 13.90it/s]


###Reading

We now have a bunch of descriptions for the websites in our training data. How do we map this to a meaningful feature representation? As suggested above, we use the approach of assigning each feature index a specific word from the descriptions, and the feature value for each site is just the count of that specific word in its description. This is just an automatic version of the keyword-based approach we used yesterday.

How do we choose which words to include as features? We could include all of them, but this would give us a lot of features for words that don't show up often enough to be helpful. Instead, we choose the 300 most frequent words.

Below, we use the CountVectorizer class from scikit-learn to do the heavy-lifting for us. We train it using just the train data (so it learns which are the 300 most frequent words in train only), and then we use to featurize both the train and val data. 


## Exercise 

Fill in the last **two lines of code** that vectorizes the val data descriptions, using the train version for reference (~4 minutes). You should save the values to *bow_val_X* and *bow_val_y*.

In [ ]:
vectorizer = CountVectorizer(max_features=300)

vectorizer.fit(train_descriptions)

def vectorize_data_descriptions(descriptions, vectorizer):
  X = vectorizer.transform(descriptions).todense()
  return X

print('\nPreparing train data...')
bow_train_X = vectorize_data_descriptions(train_descriptions, vectorizer)
bow_train_y = [label for url, html, label in train_data]
print('\nPreparing val data...')
### YOUR CODE HERE ###
bow_val_X = vectorize_data_descriptions(val_descriptions, vectorizer)
bow_val_y = [label for url, html, label in val_data]

### END CODE HERE ###


Preparing train data...

Preparing val data...


Now we have all we need to test our bag-of-words featurization. Below, we want to use logistic regression, as before, combined with our *train_X* produced by CountVectorizer to train our fake news classification model. We also want to evaluate using our familiar metrics.

## Exercise 

Fill in the code below that fits the model on *bow_train_X* and *bow_train_y* and outputs train accuracy, val accuracy, val confusion matrix, and val precision, recall, and F1-Score (~10 minutes).

In [ ]:
model = LogisticRegression()

### YOUR CODE HERE ###
model.fit(bow_train_X, bow_train_y)
bow_train_y_pred = model.predict(bow_train_X)
print('Train accuracy', accuracy_score(bow_train_y, bow_train_y_pred))


bow_val_y_pred = model.predict(bow_val_X)
print('Val accuracy', accuracy_score(bow_val_y, bow_val_y_pred))

print('Confusion matrix:')
print(confusion_matrix(bow_val_y, bow_val_y_pred))

prf = precision_recall_fscore_support(bow_val_y, bow_val_y_pred)

print('Precision:', prf[0][1])
print('Recall:', prf[1][1])
print('F-Score:', prf[2][1])
### END CODE HERE ###

Train accuracy 0.8746253746253746
Val accuracy 0.6634304207119741
Confusion matrix:
[[ 77  91]
 [ 13 128]]
Precision: 0.5844748858447488
Recall: 0.9078014184397163
F-Score: 0.7111111111111111


Solid! You should be getting results roughly similar to what you got yesterday, which may be surprising since we are restricting ourselves to look only at the description of a website. The strength of our approach today is that the featurization is automatic–we didn't put any work into determining the features.

You might be wondering how well a model that combines both our bag-of-words approach, our keywords approach, and our domain name extension approach does. We will implement combining our featurization approaches tomorrow! For now, let's explore another approach.

## Instructor-Led Discussion: Modeling the Meaning of Websites using Word Vectors


A shortcoming of our bag-of-words approach is that it only looks at the counts of words in the description for each website. What if we had some way of understanding the meaning of words in the description for each website?

The idea of computationally extracting meaning from words is central to word vectors, which have become a cornerstone of modern deep learning on text. Word vectors are a mapping from words to vectors such that words that have similar meaning have similar word vectors. 

For example, the words "good" and "great" have similar word vectors, and the words "good" and "planet" have different word vectors. Thus, word vectors provide us a way to account for the meanings of words with our machine learning models.

Run the below cell to load our word vectors, which come from a model called "GloVe".



In [ ]:
VEC_SIZE = 300
glove = GloVe(name='6B', dim=VEC_SIZE)

# Returns word vector for word if it exists, else return None.
def get_word_vector(word):
    try:
      return glove.vectors[glove.stoi[word.lower()]].numpy()
    except KeyError:
      return None

We've included a handy helper function which retrieves the word vector for a word. 

## Exercise 

Let's retrieve the word vector for "good" using the above get_word_vector function (~30 seconds).

In [ ]:
### YOUR CODE HERE ###
good_vector = get_word_vector("good")
### END CODE HERE ###

print('Shape of good vector:', good_vector.shape)
print(good_vector)

Shape of good vector: (300,)
[-1.3602e-01 -1.1594e-01 -1.7078e-02 -2.9256e-01  1.6149e-02  8.6472e-02
  1.5759e-03  3.4395e-01  2.1661e-01 -2.1366e+00  3.5278e-01 -2.3909e-01
 -2.2174e-01  3.6413e-01 -4.5021e-01  1.2104e-01 -1.5596e-01 -3.8906e-02
 -2.9419e-03  1.6009e-02 -1.1620e-01  3.8680e-01  3.5109e-01  9.7426e-02
 -1.2425e-02 -1.7864e-01 -2.3259e-01 -2.6960e-01  4.1083e-02 -7.6194e-02
 -2.3362e-01  2.0919e-01 -2.7264e-01  5.4967e-02 -1.8055e+00  5.6348e-01
 -1.2778e-01  2.3147e-01 -5.8820e-03 -2.6630e-01  4.1187e-01 -3.7162e-01
 -2.0600e-01 -1.9619e-01 -4.3945e-03  1.2513e-01  4.6638e-01  4.5159e-01
 -1.5000e-01  5.9589e-03  5.9070e-02 -4.1440e-01  6.1035e-02 -2.1117e-01
 -4.0988e-01  5.6393e-01  2.3021e-01  2.7240e-01  4.9364e-02  1.4239e-01
  4.1841e-01 -1.3983e-01  3.4826e-01 -1.0745e-01 -2.5002e-01 -3.2554e-01
  3.3343e-01 -3.5617e-01  2.0442e-01  1.4439e-01 -1.2686e-01 -7.7273e-02
 -1.9667e-01  1.0759e-01 -1.1860e-01 -2.5083e-01  1.4205e-02  2.7251e-01
 -2.3707e-01 -2.3545e-

### Reading

Not too much to see here–each word vector is a vector of 300 numbers, and it's hard to interpret them from looking at the numbers. Remember that the important property of word vectors is that words with similar meaning have similar word vectors. The magic happens when we compare word vectors.

Below, we have set up a demo where we compare the word vectors for two words using a comparison metric known as cosine similarity. Intuitively, cosine similarity measures the extent to which two vectors point in the same direction. You might be familiar with the fact that the cosine similarity between two vectors is the same as the cosine of the angle between the two vectors–ranging between -1 and 1. -1 means that two vectors are facing opposite directions, 0 means that they are perpindicular, and 1 means that they are facing the same direction. 


## Instructor-Led Discussion: Comparing Word Similarities

Try running the below to compare the vectors for "good" and "great", and then try other words, like "planet". **What do you notice that's expected and unexpected? Play around for a couple of minutes then discuss as a class.**

Note that the demo runs automatically when you change either *word1* or *word2*.

In [ ]:
#@title Word Similarity { run: "auto", display-mode: "both" }

def cosine_similarity(vec1, vec2):    
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

word1 = "frog" #@param {type:"string"}
word2 = "frogs" #@param {type:"string"}

print('Word 1:', word1)
print('Word 2:', word2)

def cosine_similarity_of_words(word1, word2):
  vec1 = get_word_vector(word1)
  vec2 = get_word_vector(word2)
  
  if vec1 is None:
    print(word1, 'is not a valid word. Try another.')
  if vec2 is None:
    print(word2, 'is not a valid word. Try another.')
  if vec1 is None or vec2 is None:
    return None
  
  return cosine_similarity(vec1, vec2)
  

print('\nCosine similarity:', cosine_similarity_of_words(word1, word2))


Word 1: frog
Word 2: frogs

Cosine similarity: 0.6233975


### Reading

We can see that word embeddings appear to capture the meaning of different words–when two words are similar, the cosine similarity score is higher, and when two words are dissimilar, the cosine similarity score is lower.

Word vectors are created by going over a large body of text (the vectors you are using were trained on Wikipedia in part) and noticing which words tend to occur near each-other. If word A tends to co-occur with similar words as word B, then the word vectors for words A and B are mathematically constrained to be similar. If you want to learn more about an algorithm for training word vectors, see this [helpful introduction to word2vec](https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa).

Given word vectors that represent the meaning of words, what can we do with this? We can add word vectors to our feature vector, but which do we choose? It turns out that a solid approach is just to average the word vectors for all the words in the description. Averaging word vectors produces a natural way to produce vectors for sentences and other collections of words, and this is the approach we will use.

## Exercise 

We want to write a function that takes a list of descriptions and turns it into an array containing the average GloVe vector for each description. **Understand the code below and then increment found_words and add vec to X[i].**

In [ ]:
def glove_transform_data_descriptions(descriptions):
    X = np.zeros((len(descriptions), VEC_SIZE))
    for i, description in enumerate(descriptions):
        found_words = 0.0
        description = description.strip()
        for word in description.split(): 
            vec = get_word_vector(word)
            if vec is not None:
                ### YOUR CODE HERE ###
                # Increment found_words and add vec to X[i].
                found_words += 1
                X[i] += vec
                
                ### END CODE HERE ###
        # We divide the sum by the number of words added, so we have the
        # average word vector.
        if found_words > 0:
            X[i] /= found_words
            
    return X
  
glove_train_X = glove_transform_data_descriptions(train_descriptions)
glove_train_y = [label for (url, html, label) in train_data]

glove_val_X = glove_transform_data_descriptions(val_descriptions)
glove_val_y = [label for (url, html, label) in val_data]

## Exercise 

Then, we can evaluate our approach as we have in the past. As before, fill in the code for fitting and evaluation (~8 minutes).

In [ ]:
model = LogisticRegression()
### YOUR CODE HERE ###
model.fit(glove_train_X, glove_train_y)
glove_train_y_pred = model.predict(glove_train_X)
print('Train accuracy', accuracy_score(glove_train_y, glove_train_y_pred))


glove_val_y_pred = model.predict(glove_val_X)
print('Val accuracy', accuracy_score(glove_val_y, glove_val_y_pred))

print('Confusion matrix:')
print(confusion_matrix(glove_val_y, glove_val_y_pred))

prf = precision_recall_fscore_support(glove_val_y, glove_val_y_pred)

print('Precision:', prf[0][1])
print('Recall:', prf[1][1])
print('F-Score:', prf[2][1])
### END CODE HERE ###

Train accuracy 0.8656343656343657
Val accuracy 0.7702265372168284
Confusion matrix:
[[116  52]
 [ 19 122]]
Precision: 0.7011494252873564
Recall: 0.8652482269503546
F-Score: 0.7746031746031746


We can see that we again get solid results using a different approach. Each approach is encoding different information about websites, and so we would expect that combining them together would produce even better results. 

## Exercise 

Fill in *train_and_evaluate_model*, which trains and evaluates a logistic regression model given train_X, train_y, val_X, and val_y. Print train accuracy, val accuracy, confusion matrix, precision, recall, and F1-score, just as we have been doing the past few days (~7 minutes).

In [ ]:
def train_model(train_X, train_y, val_X, val_y):
  model = LogisticRegression(solver='liblinear')
  model.fit(train_X, train_y)
  
  return model


def train_and_evaluate_model(train_X, train_y, val_X, val_y):
  model = train_model(train_X, train_y, val_X, val_y)
  
  ### YOUR CODE HERE ###
  model.fit(train_X, train_y)
  train_y_pred = model.predict(train_X)
  print('Train accuracy', accuracy_score(train_y, train_y_pred))
 

  val_y_pred = model.predict(val_X)
  print('Val accuracy', accuracy_score(val_y, val_y_pred))

  print('Confusion matrix:')
  print(confusion_matrix(val_y, val_y_pred))

  prf = precision_recall_fscore_support(val_y, val_y_pred)

  print('Precision:', prf[0][1])
  print('Recall:', prf[1][1])
  print('F-Score:', prf[2][1])
  ### END CODE HERE ###
  
  return model

### Review of story so far

Our first approach to featurizing our data involved looking only at URLs, specifically domain name extensions. We discovered that this achieved about 60% accuracy, with many false negatives for fake news websites with an innocuous domain name extension like ".com". This accuracy wasn't great, but it gave us a baseline to improve upon.

We next moved on to keyword-based featurization, which combined the above domain name extension features with normalized counts (extracted from HTML) for a list of specific words. 


## Exercise 

Given the functions for featurizing data before and our new helper function *train_and_evaluate_model*, train and evaluate a model on top of keyword (combined with domain name extension) featurization (~8 minutes).

In [ ]:
def prepare_data(data, featurizer):
    X = []
    y = []
    for datapoint in data:
        url, html, label = datapoint
        # We convert all text in HTML to lowercase, so <p>Hello.</p> is mapped to
        # <p>hello</p>. This will help us later when we extract features from 
        # the HTML, as we will be able to rely on the HTML being lowercase.
        html = html.lower() 
        y.append(label)

        features = featurizer(url, html)

        # Gets the keys of the dictionary as descriptions, gets the values
        # as the numerical features. Don't worry about exactly what zip does!
        feature_descriptions, feature_values = zip(*features.items())

        X.append(feature_values)

    return X, y, feature_descriptions
  
# Gets the log count of a phrase/keyword in HTML (transforming the phrase/keyword
# to lowercase).
def get_normalized_count(html, phrase):
    return math.log(1 + html.count(phrase.lower()))

# Returns a dictionary mapping from plaintext feature descriptions to numerical
# features for a (url, html) pair.
def keyword_featurizer(url, html):
    features = {}
    
    # Same as before.
    features['.com domain'] = url.endswith('.com')
    features['.org domain'] = url.endswith('.org')
    features['.net domain'] = url.endswith('.net')
    features['.info domain'] = url.endswith('.info')
    features['.org domain'] = url.endswith('.org')
    features['.biz domain'] = url.endswith('.biz')
    features['.ru domain'] = url.endswith('.ru')
    features['.co.uk domain'] = url.endswith('.co.uk')
    features['.co domain'] = url.endswith('.co')
    features['.tv domain'] = url.endswith('.tv')
    features['.news domain'] = url.endswith('.news')
    
    keywords = ['trump', 'biden', 'clinton', 'sports', 'finance']
    
    for keyword in keywords:
      features[keyword + ' keyword'] = get_normalized_count(html, keyword)
    
    return features


### YOUR CODE HERE ###
keyword_train_X, train_y,_= prepare_data(train_data, keyword_featurizer)
keyword_val_X, val_y,_ = prepare_data(val_data, keyword_featurizer)

train_and_evaluate_model(keyword_train_X, train_y,keyword_val_X, val_y)


### END CODE HERE ###

Train accuracy 0.7922077922077922
Val accuracy 0.7346278317152104
Confusion matrix:
[[106  62]
 [ 20 121]]
Precision: 0.6612021857923497
Recall: 0.8581560283687943
F-Score: 0.7469135802469136


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

We can see that we are achieving significantly better accuracy and better balance between false negatives and false positives. As expected, it looks like actually making use of the content of the HTML is useful.

Another way to take advantage of the HTML is to extract a bag-of-words (BOW) featurization from the meta description stored in the HTML. 

## Instructor-Led Discussion: Bag of Words featurization

## Exercise 

As before, train and evaluate this approach with our helper function *train_and_evaluate_model* (~5 minutes).

In [ ]:
vectorizer = CountVectorizer(max_features=300)

vectorizer.fit(train_descriptions)

def vectorize_data_descriptions(data_descriptions, vectorizer):
  X = vectorizer.transform(data_descriptions).todense()
  return X

### YOUR CODE HERE ###

# Note that you can use train_y and val_y from before, since these are the
# same for both the keyword approach and the BOW approach.

bow_train_X = vectorize_data_descriptions(train_descriptions, vectorizer)
bow_val_X = vectorize_data_descriptions(val_descriptions, vectorizer)

train_and_evaluate_model(bow_train_X, train_y,bow_val_X, val_y)



### END CODE HERE ###

Train accuracy 0.8746253746253746
Val accuracy 0.6634304207119741
Confusion matrix:
[[ 77  91]
 [ 13 128]]
Precision: 0.5844748858447488
Recall: 0.9078014184397163
F-Score: 0.7111111111111111


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

We can see that we are getting similar results, without necessarily using the same information as the keywords-based approach. We then asked whether we could do better by making use of word vectors, which encode the meaning of different words. We found that averaging the word vectors for words in the meta description was a useful approach. 

## Exercise 

As before, review the approach and complete the code for training and evaluation (~5 minutes).

In [ ]:
VEC_SIZE = 300
glove = GloVe(name='6B', dim=VEC_SIZE)

# Returns word vector for word if it exists, else return None.
def get_word_vector(word):
    try:
      return glove.vectors[glove.stoi[word.lower()]].numpy()
    except KeyError:
      return None

def glove_transform_data_descriptions(descriptions):
    X = np.zeros((len(descriptions), VEC_SIZE))
    for i, description in enumerate(descriptions):
        found_words = 0.0
        description = description.strip()
        for word in description.split(): 
            vec = get_word_vector(word)
            if vec is not None:
                ### YOUR CODE HERE ###
                # Increment found_words and add vec to X[i].
                found_words += 1
                X[i] += vec
                ### END CODE HERE ###
        # We divide the sum by the number of words added, so we have the
        # average word vector.
        if found_words > 0:
            X[i] /= found_words
            
    return X
  
  
### YOUR CODE HERE ###

# Note that you can use train_y and val_y from before, since these are the
# same for both the keyword approach and the BOW approach.
glove_train_X = glove_transform_data_descriptions(train_descriptions)
glove_val_X = glove_transform_data_descriptions(val_descriptions)

train_and_evaluate_model(glove_train_X, train_y,glove_val_X, val_y)


### END CODE HERE ###

Train accuracy 0.8656343656343657
Val accuracy 0.7702265372168284
Confusion matrix:
[[116  52]
 [ 19 122]]
Precision: 0.7011494252873564
Recall: 0.8652482269503546
F-Score: 0.7746031746031746


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

Again, solid results using a completely different approach.

### Instructor-Led Discussion: Combining Approaches

A natural question to ask now is whether we can combine the above featurization approaches for improved results. It turns out we can, by concatenating the feature vectors for each website produced using each of the three above approaches. Below we provide a handy helper function that takes in a list of multiple train_X produced using different featurization approaches (e.g., [keyword_train_X, bow_train_X, glove_train_X]) and combines them into *combined_train_X*. It can do this for val_X as well.

As an example, if our keyword-based approach has 15 features, our BOW approach has 300, and our GloVe approach has 300, then our combined approach has 615 features.


## Exercise 

Complete the below code for training and evaluating the combined approach (~8 minutes).

In [ ]:
def combine_features(X_list):
  return np.concatenate(X_list, axis=1)

### YOUR CODE HERE ###
# First, produce combined_train_X and combined_val_X using 2 calls to 
# combine_features, using keyword_train_X, bow_train_X, glove_train_X
# and keyword_val_X, bow_val_X, glove_val_X from before.
combined_train_X = combine_features([keyword_train_X, bow_train_X, glove_train_X])
combined_val_X = combine_features([keyword_val_X, bow_val_X, glove_val_X]) 

train_and_evaluate_model(combined_train_X, train_y,combined_val_X, val_y)


### END CODE HERE ###

Train accuracy 0.9175824175824175
Val accuracy 0.7993527508090615
Confusion matrix:
[[119  49]
 [ 13 128]]
Precision: 0.7231638418079096
Recall: 0.9078014184397163
F-Score: 0.8050314465408805


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

## Exercise 

Now, make changes to the keyword, BOW, and GloVe featurization code above to improve performance. For example, add keywords to the keyword featurization code, play around with different values for *max_features* and other parameters for [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), and try summing instead of averaging word vectors (~15 minutes).

## Exercise 

Once you feel satisfied with your results, play around with the below demo, which scrapes a new website live and runs your trained classification model on it! The below code assumes you have run the code above and have not changed the names of any of the featurization functions. It combines the three featurization approaches. Go through the code below and make sure you understand what it is doing! Feel free to also make changes (e.g. if you only want to use two featurization approaches) (~15 minutes). Note that output is shown below the *curr_url* field.

In [ ]:
#@title Live Fake News Classification Demo { run: "auto", vertical-output: true, display-mode: "both" }
def get_data_pair(url):
  if not url.startswith('http'):
      url = 'http://' + url
  url_pretty = url
  if url_pretty.startswith('http://'):
      url_pretty = url_pretty[7:]
  if url_pretty.startswith('https://'):
      url_pretty = url_pretty[8:]
      
  # Scrape website for HTML
  response = requests.get(url, timeout=10)
  htmltext = response.text
  
  return url_pretty, htmltext

curr_url = "https://www.cnn.com" #@param {type:"string"}

url, html = get_data_pair(curr_url)

# Call on the output of *keyword_featurizer* or something similar
# to transform it into a format that allows for concatenation. See
# example below.
def dict_to_features(features_dict):
  X = np.array(list(features_dict.values())).astype('float')
  X = X[np.newaxis, :]
  return X
def featurize_data_pair(url, html):
  # Approach 1.
  keyword_X = dict_to_features(keyword_featurizer(url, html))
  # Approach 2.
  description = get_description_from_html(html)
  
  bow_X = vectorize_data_descriptions([description], vectorizer)
  
  # Approach 3.
  glove_X = glove_transform_data_descriptions([description])
  
  X = combine_features([keyword_X, bow_X, glove_X])
  
  return X

curr_X = featurize_data_pair(url, html)

model = train_model(combined_train_X, train_y, combined_val_X, val_y)

curr_y = model.predict(curr_X)[0]
  
  
if curr_y < .5:
  print(curr_url, 'appears to be real.')
else:
  print(curr_url, 'appears to be fake.')

https://www.cnn.com appears to be real.


## Exercise 

After playing around with your model live, do you have any ideas for how to improve it? 

1. Write down three ideas for how to improve it. 

2. Then, the make changes above and see what impact they have on the final model.

Once you are done, we will provide code to test your model on the test data. Note that you should only run on the test data once. When you have results on the test data, share with your instructor, as the team with the best results will be rewarded!

In [ ]:
### PUT TEST CODE HERE ###
with open(os.path.join(basepath, 'test_data.pkl'), 'rb') as f:
  test_data = pickle.load(f)
  
model = train_model(combined_train_X, train_y, combined_val_X, val_y)
val_y_pred = model.predict(combined_val_X)
print('Val accuracy', accuracy_score(val_y, val_y_pred))

print('Loading test data...')
test_X = []
for url, html, label in test_data:
  curr_X = np.array(featurize_data_pair(url, html))
  test_X.append(curr_X[0])
  
test_X = np.array(test_X)

test_y = [label for url, html, label in test_data]

print('Done loading test data...')

test_y_pred = model.predict(test_X)

print('Test accuracy', accuracy_score(test_y, test_y_pred))

print('Confusion matrix:')
print(confusion_matrix(test_y, test_y_pred))

prf = precision_recall_fscore_support(test_y, test_y_pred)

print('Precision:', prf[0][1])
print('Recall:', prf[1][1])
print('F-Score:', prf[2][1])

### END CODE HERE ###

NameError: ignored

A huge congratulations on completing the project!